This is a simple/naive pipeline to show how to utilize fastai and ULMFIT in order to tackle this problem: https://docs.fast.ai/tutorial.text.html

Inference notebook can be found here: https://www.kaggle.com/alibaba19/jigsaw-ulmfit-fastai-inference/notebook

In [ ]:
from fastai.text.all import *

In [ ]:
old_jigsaw_path = Path('../input/jigsaw-toxic-comment-classification-challenge')
input_path = Path('../input/jigsaw-toxic-severity-rating')

In [ ]:
!unzip ../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip

In [ ]:
df = pd.read_csv('../working/train.csv')
df.shape

In [ ]:
df.tail(3)

In [ ]:
df[df.toxic==1].head(4)

In [ ]:
df['total_tox'] = df[df.columns[2:]].sum(axis=1)

In [ ]:
df['total_tox'].value_counts()

So most of them don't have any toxicity, which we can score as zero. Only 31 are maximally toxic on our arbitrary scale. Probably need to be more mindful about the data collection and parsing, but we'll use this for now to proceed

In [ ]:
db_lm = DataBlock(
            blocks = TextBlock.from_df('comment_text', is_lm=True),
    get_x =ColReader('text'),       
    splitter=RandomSplitter(0.1),
    ).dataloaders(df, bs=64, seq_len=80)

In [ ]:
db_lm.show_batch(max_n=1)

In [ ]:
vocab = pd.Series(db_lm.vocab)
vocab.to_csv('lm_vocab.csv')

In [ ]:
learn = language_model_learner(
            db_lm, AWD_LSTM, drop_mult=0.3,
            metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1,2e-2)

In [ ]:
learn.save_encoder('lm_encoder')

In [ ]:
toxic_db = DataBlock(blocks=(TextBlock.from_df('comment_text', vocab=db_lm.vocab), RegressionBlock),
                     get_x = ColReader('text'),
                     get_y = ColReader('total_tox'),
                     splitter = RandomSplitter())

In [ ]:
dls = toxic_db.dataloaders(df, bs=64)
dls.show_batch(max_n=1)

In [ ]:
learner = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learner = learner.load_encoder('lm_encoder')

In [ ]:
learner.fine_tune(4, 1e-2)

In [ ]:
learner.show_results()

In [ ]:
learner.save('initial_model')